<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/hair_styling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import xml.etree.ElementTree as ET
import os
import shutil

In [13]:
# move images and corresponding segmentation masks into their own folders
# project_path = '/content/drive/My Drive/hairy_gan/'
# dataset_path = os.path.join(project_path, 'face_segment')
# src_img_target_path = os.path.join(dataset_path, 'images')
# label_img_target_path = os.path.join(dataset_path, 'masks')
# tree = ET.parse(os.path.join(dataset_path, 'sampleset.xml'))
# root = tree.getroot()
# children = root.getchildren()
# for i in range(len(children) // 2):
#   src_img_path = children[2 * i].get('name').replace('\\', '/')
#   label_img_path = children[2 * i + 1].get('name').replace('\\', '/')
  
#   shutil.copy(os.path.join(dataset_path, src_img_path), os.path.join(src_img_target_path, os.path.basename(src_img_path)))
#   shutil.copy(os.path.join(dataset_path, label_img_path), os.path.join(label_img_target_path, os.path.basename(label_img_path)))

206